# Trabajo de Diseño de patrones y diseño de software - Patrones creacionales

### Integrantes: 
 #### - Wilson Andrés Vargas Rojas
 #### - Yeison Fabian Suarez Alba 

## Punto 2
Se ha creado un programa para la generación de documentos sin importar su formato a partir de plantillas, para ello nos hemos basado en tres patrones creacionales, Prototype, Factory Method y Builder, que los iremos exponiendo durante el trabajo.

## Trabajar a partir de plantillas, el patrón Prototype.
Para poder crear una copia de la plantilla se usará el patrón Prototype, que nos ayuda a crear un nuevo objeto documento a partir de uno ya existente.

### IPrototype
Es una interfaz con un único método para realizar la clonación.

In [163]:
public interface IPrototype
    {
        IPrototype Clone();
    }

### DocumentFile
Esta es la clase abstracta que heredará los metodos mas relevantes para la creación de los documentos, ya que algunos métodos se comparten entre sí pero cada uno tiene que proporcionar su propia lógica, Haciendo que los principios SOLID se respeten.

In [164]:
//Se añade el método para realizar el Prototype y se implementa la interfaz
public abstract class DocumentFile: IPrototype
    {
        public string FileName { get; set; } 
        public string Title { get; set; }
        public List<string> Columns { get; set; } = new List<string>();
        public List<string> Tables { get; set; } = new List<string>();
    
        public abstract void AddColumn(string column);
        public abstract void AddTable(string tableData);
        public abstract void SetTitle(string title);
        public abstract string RetrieveContent(); 

        public abstract IPrototype Clone();
    }

### Clases especificas (DOCXDocumentFile, XLSXDocumentFile, XMLDocumentFile)
Estos son nuestros productos especificos

In [165]:
public class DOCXDocumentFile : DocumentFile
    {
        public DOCXDocumentFile(string fileName) 
        {
            FileName = fileName;
        }
    
        public override void AddColumn(string column)
        {
            Console.WriteLine(String.Format("Añadiendo columna {0}...",column));
            Columns.Add(column);
            
        }
    
        public override void AddTable(string tableData)
        {
            Console.WriteLine(String.Format("Añadiendo tabla {0}...",tableData));
            Tables.Add(tableData);
        }
    
        public override void SetTitle(string title)
        {
            Console.WriteLine(String.Format("Añadiendo el titulo {0}...",title));
            Title = title;
        }
    
        public override string RetrieveContent()
        {
            Console.WriteLine("Archivo Word: ");
            Console.WriteLine(String.Format("Titulo: {0}", Title));
            Console.WriteLine(String.Format("Columnas: {0}", String.Join(",", Columns)));
            Console.WriteLine(String.Format("Tablas: {0}", String.Join(",", Tables)));
            return "Contenido del archivo...";
        }

        public override IPrototype Clone()
        {
            return new DOCXDocumentFile(this.FileName);
        }
    
}

In [166]:
public class XLSXDocumentFile : DocumentFile
    {

        public XLSXDocumentFile(string fileName)
        {
            FileName = fileName;
        }
    
        public override void AddColumn(string column)
        {
            Console.WriteLine(String.Format("Añadiendo la columna {0}...",column));
            Columns.Add(column);
        }
    
        public override void AddTable(string tableData)
        {
            Console.WriteLine(String.Format("Añadiendo la tabla {0}...",tableData));
            Tables.Add(tableData);
        }
    
        public override void SetTitle(string title)
        {
            Console.WriteLine(String.Format("Añadiendo el titulo {0}...",title));
            Title = title;
        }
    
        public override string RetrieveContent()
        {
            Console.WriteLine("Archivo Excel: ");
            Console.WriteLine(String.Format("Titulo: {0}", Title));
            Console.WriteLine(String.Format("Columnas: {0}", String.Join(",", Columns)));
            Console.WriteLine(String.Format("Tablas: {0}", String.Join(",", Tables)));
            return "Contenido del archivo...";
        }
    
        public override IPrototype Clone()
        {
            return new XLSXDocumentFile(this.FileName);
        }
    }

In [167]:
public class XMLDocumentFile : DocumentFile
    {
        public XMLDocumentFile(string fileName)
        {
            FileName = fileName;
        }

        public override void AddColumn(string column)
        {
            Console.WriteLine(String.Format("Añadiendo la columna {0}...",column));
            Columns.Add("<Column>" + column + "</Column>");
        }
        
        public override void AddTable(string tableData)
        {
            Console.WriteLine(String.Format("Añadiendo la tabla {0}...",tableData));
            Tables.Add("<Table>" + tableData + "</Table>");
        }
        
        public override void SetTitle(string title)
        {
            Console.WriteLine(String.Format("Añadiendo el titulo {0}...",title));
            Title = "<Title>" + title + "</Title>";
        }
        
        public override string RetrieveContent()
        {
            Console.WriteLine("Archivo XML");
            Console.WriteLine(String.Format("Titulo: {0}", Title));
            Console.WriteLine(String.Format("Columnas: {0}", String.Join(",", Columns)));
            Console.WriteLine(String.Format("Tablas: {0}", String.Join(",", Tables)));
            return "Contenido del archivo...";
        }

        public override IPrototype Clone()
        {
            return new XMLDocumentFile(this.FileName);
        }
    }

Ahora, si quiero trabajar a partir de una plantilla tendría:

In [168]:
//Archivo plantilla que existe
DocumentFile ExistingTemplateFile = new DOCXDocumentFile("documentoprueba.docx");

//Generar un archivo a partir de la plantilla
DocumentFile FileByTemplate = (DocumentFile)ExistingTemplateFile.Clone();
//Cambiar los atributos especificos de la plantilla
FileByTemplate.SetTitle("Mi trabajo");
FileByTemplate.AddColumn("Reporte");
FileByTemplate.AddColumn("Grafica");

FileByTemplate.RetrieveContent();

Añadiendo el titulo Mi trabajo...
Añadiendo columna Reporte...
Añadiendo columna Grafica...
Archivo Word: 
Titulo: Mi trabajo
Columnas: Reporte,Grafica
Tablas: 


## Crear nuevos documentos, el patrón Factory Method

Al momento de crear un documento basta con instanciar el objeto, pero que pasa si se cambia la manera en que se construye el documento?, tendría que ir a cambiar cada instanciación, rompiendo el Principio Open-Close de los principios SOLID, para ello hacermos uso del patrón Factory Method.

### La fabrica: 

In [169]:
public abstract class DocumentFileFactory
    {
        public abstract DocumentFile CreateDocumentFile(String filename);
    }

### Fabricas concretas: XMLDocumentFactory, XLSXDocumentFactory, DOCXDocumentFactory
Cada fabrica concreta me creará el documento según el tipo necesario.

In [170]:
//Fabrica para Excel
public class XLSXDocumentFactory : DocumentFileFactory
    {
        public override DocumentFile CreateDocumentFile(String filename)
        {
            return new XLSXDocumentFile(filename);
        }
    }

//Fabrica para Word
public class DOCXDocumentFactory : DocumentFileFactory
    {
        public override DocumentFile CreateDocumentFile(String filename)
        {
            return new DOCXDocumentFile(filename);
        }
    }

//Fabrica para XML
public class XMLDocumentFactory : DocumentFileFactory
    {
        public override DocumentFile CreateDocumentFile(String filename)
        {
            return new XMLDocumentFile(filename);
        }
    }

Ahora cuando se quiera crear un documento crearemos y llamaremos a las fábricas.

In [171]:
// Crear instancias de las fábricas
XLSXDocumentFactory excelFactory = new XLSXDocumentFactory();
XMLDocumentFactory xmlFactory = new XMLDocumentFactory();
DOCXDocumentFactory wordFactory = new DOCXDocumentFactory();

// Crear documentos a través de las fábricas
DocumentFile excelDocument = excelFactory.CreateDocumentFile("ExcelDocumento.xlsx");
DocumentFile xmlDocument = xmlFactory.CreateDocumentFile("XMLDocumento.xml");
DocumentFile wordDocument = wordFactory.CreateDocumentFile("WordDocumento.docx");

## Crear documentos paso a paso - El patrón Builder
Al crear los documentos podríamos encontrarnos con que crearlos desde cero puede ser muy laborioso con un contructor bastante extenso y dificil de manejar. Con el patrón builder podremos crear lo necesario paso a paso.

### La Interfaz Builder
Esta interfaz va tener todos los métodos para todos aquellos atributos que quiero utilizar en la construcción de mi documento, logrando también la segregación de interfaces.

In [172]:
public interface IBuilder
{
    IBuilder SetTitle(string title); 
    IBuilder AddTable(string tableData);
    IBuilder AddColumn(string column);
    IBuilder ApplyStyle(string style);
    DocumentFile Build(); 
}

Ahora bien, como queremos crear un PDF, debemos definirle una clase que herede de DocumentFile, sin emabrgo, al heredar directamente tendriamos dos problemas, uno de ellos es que habrían métodos que no puede usar un PDF en DocumentFIle, violando el principio de Liskov

### El builder concreto para crear mi documento

In [173]:
public class DocumentBuilder : IBuilder
{
    private DocumentFile document;
    
    public DocumentBuilder(DocumentFile document) {
        this.document = document;  // Corrección: asignar el parámetro al campo de instancia.
    }
    
    public IBuilder SetTitle(string title) {
        document.SetTitle(title);
        return this;
    }
    
    public IBuilder AddColumn(string column) {
        document.AddColumn(column);
        return this;
    }
    
    public IBuilder AddTable(string tableData) {
        document.AddTable(tableData);
        return this;
    }
    
    public IBuilder ApplyStyle(string style) {
        // Implementar la aplicación de estilo si es necesario
        return this;
    }
    
    public DocumentFile Build() {
        return document;
    }
}


Ahora podremos contruir el documento como lo requeramos:

In [174]:
DocumentFile docxDoc = new DOCXDocumentFile("reporte.docx");
IBuilder builder = new DocumentBuilder(docxDoc);
DocumentFile doc = builder
    .SetTitle("Reporte mensual")
    .AddColumn("Balance anual")
    .AddColumn("Sucursal")
    .AddTable("Ventas a clientes")
    .Build();

doc.RetrieveContent();

Añadiendo el titulo Reporte mensual...
Añadiendo columna Balance anual...
Añadiendo columna Sucursal...
Añadiendo tabla Ventas a clientes...
Archivo Word: 
Titulo: Reporte mensual
Columnas: Balance anual,Sucursal
Tablas: Ventas a clientes


## Convertir a PDF - Juntando todo
Luego de haber creado tres patrones creacionales, ya podemos hacer uso de todo el contenido para crear nuestro PDF.

In [197]:
public class PdfDocument
{
    String FileName;
    
    public PdfDocument(string fileName) {
        FileName = fileName;
    }

    public string RetrieveContent() {
        return "Contenido PDF";
    }
    
    // Método estático para simular la generación real del PDF a partir de un DocumentFile
    public static PdfDocument GeneratePdf(DocumentFile document) {
        PdfDocument pdf = new PdfDocument(document.FileName);
        Console.WriteLine("PDF Generado.");
        return pdf;
    }
}

Ahora empecemos con la creación de un ejemplo:

In [198]:
//El usuario decidio crear un Word, se llama a la fabrica concreta
DocumentFileFactory factory = new DOCXDocumentFactory();

//Se crea el archivo Word
DocumentFile file = factory.CreateDocumentFile("Notas del semestre.docx");

//Se crea el builder para el documento
IBuilder Mybuilder = new DocumentBuilder(file);

//Crea un template
DocumentFile template = Mybuilder
    .SetTitle("Notas")
    .AddColumn("Semestre 1")
    .AddTable("Promedio de notas")
    .Build();

Console.WriteLine("Plantilla: ");
//Contenido del template
template.RetrieveContent();

//Crear una copia del template
DocumentFile myReport = (DocumentFile)template.Clone();

Console.WriteLine("Mi archivo: ");
//Modificar el archivo
myReport = Mybuilder
    .SetTitle("Mis notas")
    .AddColumn("Semestre 2")
    .AddTable("Ingenieria de requisitos")
    .AddTable("Diseño de patrones")
    .Build();

//Contenido de mi archivo
myReport.RetrieveContent();

//Generar archivo PDF
PdfDocument myReportPdf = PdfDocument.GeneratePdf(myReport);

Añadiendo el titulo Notas...
Añadiendo columna Semestre 1...
Añadiendo tabla Promedio de notas...
Plantilla: 
Archivo Word: 
Titulo: Notas
Columnas: Semestre 1
Tablas: Promedio de notas
Mi archivo: 
Añadiendo el titulo Mis notas...
Añadiendo columna Semestre 2...
Añadiendo tabla Ingenieria de requisitos...
Añadiendo tabla Diseño de patrones...
Archivo Word: 
Titulo: Mis notas
Columnas: Semestre 1,Semestre 2
Tablas: Promedio de notas,Ingenieria de requisitos,Diseño de patrones
PDF Generado.
